# LLM Generated Data

We have prompted a couple of free and well know LLM's for and reproduction of additional data since we had to spend a long time finding the data ourselves. In this notebook we will compare the data and possibly merge it into our data set.

In [830]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import os
from fuzzywuzzy import process
import openpyxl
import warnings
warnings.filterwarnings("ignore")

In [831]:
# get current working directory 
cwd = os.getcwd() + "/../LLM_generated/"
# print(cwd)
df = pd.read_excel(cwd + "UniversitiesCompletePrices.xlsx")
completeDF = pd.read_csv(cwd + "../completeDatasets/pricesComplete.csv")                

We are not really satisfied with the ranking LLM's provided so we will be merging our original ranking file into here. For more information 
checkout: <a href="https://cwur.org/2024.php">World University Rankings 2024</a> or find reference in <a href="./../docs/docs.docx">UniMatch documentation.<a> 

In [832]:
rankingDF = pd.read_csv(cwd + "../fillers/uniRanking.csv")
removeColumns = ["Rank", "Country", "Teaching", "International Outlook", "Research", "Citations", "Industry Income", "Overall"]
rankingDF.drop(columns=removeColumns, errors="ignore", inplace = True)
# index is rank - 1
rankingDF["Ranking"]= rankingDF.index + 1
rankingDF.to_csv("../fillers/rankingCleaned.csv", index = False)
rankingDF.head()

,University,Ranking
0,University of Oxford,1
1,Stanford University,2
2,Massachusetts Institute of Technology,3
3,Harvard University,4
4,University of Cambridge,5


In [833]:
def normalize(name):
    return name.split(" (")[0]

df["University"] = df["University"].apply(normalize)
rankingDF["University"] = rankingDF["University"].apply(normalize)

mappings = dict()
def fuzzyMatch(name, reference_df, threshold=95):
    match = process.extractOne(name, reference_df["University"])
    if match and match[1] >= threshold:
        if name in mappings:
            mappings[name].add((match[1],match[0]))
        else :
            initSet = set()
            initSet.add((match[1],match[0]))
            mappings[name] = initSet
        # print(name,mappings[name])
        return match[0]
    return None
for x in rankingDF["University"]:
    fuzzyMatch(x, df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 435 entries, 0 to 434
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Country              435 non-null    object 
 1   University           435 non-null    object 
 2   CollegeRank          435 non-null    object 
 3   Tuition              435 non-null    int64  
 4   PercOfIntStud        435 non-null    int64  
 5   AcceptanceRate       435 non-null    int64  
 6   AvgSafetyIndex       435 non-null    int64  
 7   CostOfLiving         435 non-null    int64  
 8   Rent                 435 non-null    int64  
 9   Groceries            435 non-null    int64  
 10  RecreationCost       435 non-null    int64  
 11  HealthcarePrice      435 non-null    int64  
 12  AvgMntTransportCost  435 non-null    int64  
 13  Link                 435 non-null    object 
 14  Computer Science     435 non-null    int64  
 15  Business             435 non-null    int

In [834]:
bestMappings = dict()
for x in mappings.items():
    best = 0
    for y in x[1]:
        # print(y[1])
        if y[0] > best:
            best = y[0]
            bestMappings[x[0]] = y[1]
#print(bestMappings)
rankingDF["University"] = rankingDF["University"].replace(bestMappings)
rankingDF = rankingDF.drop_duplicates(subset="University", keep="first")
df = df.drop_duplicates(subset = "University", keep = "first")

df = pd.merge(df, rankingDF, on="University", how="left")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434 entries, 0 to 433
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Country              434 non-null    object 
 1   University           434 non-null    object 
 2   CollegeRank          434 non-null    object 
 3   Tuition              434 non-null    int64  
 4   PercOfIntStud        434 non-null    int64  
 5   AcceptanceRate       434 non-null    int64  
 6   AvgSafetyIndex       434 non-null    int64  
 7   CostOfLiving         434 non-null    int64  
 8   Rent                 434 non-null    int64  
 9   Groceries            434 non-null    int64  
 10  RecreationCost       434 non-null    int64  
 11  HealthcarePrice      434 non-null    int64  
 12  AvgMntTransportCost  434 non-null    int64  
 13  Link                 434 non-null    object 
 14  Computer Science     434 non-null    int64  
 15  Business             434 non-null    int

### Now we rename all the same columns and we which ones we are missing

In [835]:
df = df.rename(columns={"CostOfLiving": "LivingCost", "HealthcarePrice" : "HealthcareCost", "Rent": "RentCost", "Groceries": "GroceriesCost", "AvgMntTransportCost" : "TransportCost" ,"AvgSafetyIndex": "SafetyIndex"})
s1 = set(name.lower() for name in list(completeDF.columns))
s2 = set(name.lower() for name in list(df.columns))
print("Columns we had in the original data set but couldn't get from the LLM's: ", s1.difference(s2))

Columns we had in the original data set but couldn't get from the LLM's:  {'region'}


We are missing the continent of as named region. We will merge it with our original continent.csv file

In [836]:
# Adding continent factor
continent = pd.read_csv("../fillers/continent.csv")
continent = continent.rename(columns = {"name": "Country"})
continent.head()
continent.loc[continent["region"] == "Americas", "region"] = continent["sub-region"]
continent.loc[continent["region"] == "Latin America and the Caribbean", "region"] = "Southern America"
continent= continent.rename(columns={"name": "country"})
# df = pd.merge(df, continent[["Country", "region"]], on="Country", how ="left")
# df.info()

We see that we have only 295 values filled in region column while we have 435 columns otherwise. This happens because some countries are named differently in two dataset for example USA and United States of America.

### 1. Normalize names

In [837]:
df["Country"] = df["Country"].str.upper().str.strip()
continent["Country"] = continent["Country"].str.upper().str.strip()
continent["Country"] = continent["Country"].str.split(",").str[0]
# df = pd.merge(df, continent[["Country", "region"]], on="Country", how ="left")
# df.info()

We see only a small improvement. +15 entries.

### 2. Manual mapping

In [838]:
# Thank you ChatGPT for manual labor
countryMapping = {
    # United States
    "USA": "UNITED STATES",
    "US": "UNITED STATES",
    "U.S.": "UNITED STATES",
    "U.S.A.": "UNITED STATES",
    "UNITED STATES OF AMERICA": "UNITED STATES",
    "AMERICA": "UNITED STATES",

    # United Kingdom
    "UK": "UNITED KINGDOM",
    "U.K.": "UNITED KINGDOM",
    "BRITAIN": "UNITED KINGDOM",
    "GREAT BRITAIN": "UNITED KINGDOM",
    "ENGLAND": "UNITED KINGDOM",
    "SCOTLAND": "UNITED KINGDOM",
    "WALES": "UNITED KINGDOM",
    "NORTHERN IRELAND": "UNITED KINGDOM",

    # South Korea
    "REPUBLIC OF KOREA": "SOUTH KOREA",
    "KOREA, SOUTH": "SOUTH KOREA",
    "SOUTH KOREA": "SOUTH KOREA",
    "KOREA (SOUTH)": "SOUTH KOREA",

    # North Korea
    "NORTH KOREA": "NORTH KOREA",
    "KOREA, NORTH": "NORTH KOREA",
    "KOREA (NORTH)": "NORTH KOREA",

    # Russia
    "RUSSIA": "RUSSIA",
    "RUSSIAN FEDERATION": "RUSSIA",
    "USSR": "RUSSIA",
    "SOVIET UNION": "RUSSIA",

    # China
    "CHINA": "CHINA",
    "PEOPLE'S REPUBLIC OF CHINA": "CHINA",
    "PRC": "CHINA",
    "MAINLAND CHINA": "CHINA",

    # India
    "INDIA": "INDIA",
    "REPUBLIC OF INDIA": "INDIA",
    "BHARAT": "INDIA",

    # Germany
    "GERMANY": "GERMANY",
    "FEDERAL REPUBLIC OF GERMANY": "GERMANY",
    "WEST GERMANY": "GERMANY",
    "EAST GERMANY": "GERMANY",

    # France
    "FRANCE": "FRANCE",
    "REPUBLIC OF FRANCE": "FRANCE",

    # Italy
    "ITALY": "ITALY",
    "REPUBLIC OF ITALY": "ITALY",

    # Japan
    "JAPAN": "JAPAN",
    "NIPPON": "JAPAN",
    "NIHON": "JAPAN",

    # Australia
    "AUSTRALIA": "AUSTRALIA",
    "COMMONWEALTH OF AUSTRALIA": "AUSTRALIA",

    # Brazil
    "BRAZIL": "BRAZIL",
    "FEDERATIVE REPUBLIC OF BRAZIL": "BRAZIL",

    # Argentina
    "ARGENTINA": "ARGENTINA",
    "REPUBLIC OF ARGENTINA": "ARGENTINA",

    # South Africa
    "SOUTH AFRICA": "SOUTH AFRICA",
    "REPUBLIC OF SOUTH AFRICA": "SOUTH AFRICA",

    # Saudi Arabia
    "SAUDI ARABIA": "SAUDI ARABIA",
    "KINGDOM OF SAUDI ARABIA": "SAUDI ARABIA",

    # Spain
    "SPAIN": "SPAIN",
    "KINGDOM OF SPAIN": "SPAIN",

    # Portugal
    "PORTUGAL": "PORTUGAL",
    "REPUBLIC OF PORTUGAL": "PORTUGAL",

    # Netherlands
    "NETHERLANDS": "NETHERLANDS",
    "HOLLAND": "NETHERLANDS",
    "KINGDOM OF THE NETHERLANDS": "NETHERLANDS",

    # Belgium
    "BELGIUM": "BELGIUM",
    "KINGDOM OF BELGIUM": "BELGIUM",

    # Switzerland
    "SWITZERLAND": "SWITZERLAND",
    "SWISS CONFEDERATION": "SWITZERLAND",

    # Egypt
    "EGYPT": "EGYPT",
    "ARAB REPUBLIC OF EGYPT": "EGYPT",

    # Turkey
    "TURKEY": "TURKEY",
    "REPUBLIC OF TURKEY": "TURKEY",

    # Greece
    "GREECE": "GREECE",
    "HELLENIC REPUBLIC": "GREECE",

    # Austria
    "AUSTRIA": "AUSTRIA",
    "REPUBLIC OF AUSTRIA": "AUSTRIA",

    # New Zealand
    "NEW ZEALAND": "NEW ZEALAND",
    "AOTEAROA": "NEW ZEALAND",

    # Ireland
    "IRELAND": "IRELAND",
    "REPUBLIC OF IRELAND": "IRELAND",
    "EIRE": "IRELAND"
}

In [839]:
df["Country"] = df["Country"].replace(countryMapping)
continent["Country"] = continent["Country"].replace(countryMapping)
# df = pd.merge(df, continent[["Country", "region"]], on="Country", how ="left")
# df.info()

Now we see an improvement. +100 entries.

### 3. Fuzzy matching 

In [840]:
#Not effective in our case since only a couple are left
# def fuzzy(country, validCountries):
#     match, score = process.extractOne(country, validCountries)
#     return match if score > 80 else None

# valid = df["Country"].unique()
# continent["Country"] = continent["Country"].apply(lambda x: fuzzy(x, valid))
# df = pd.merge(df, continent[["Country", "region"]], on="Country", how ="left")
# df.head(35)

### 4. Actual manual labor

In [841]:
# We just add more mapping by hand this time and insert into our continent.csv
manualMapping = {
    "KOREA" : "SOUTH KOREA",
    "UNITED KINGDOM OF GREAT BRITAIN AND NORTHERN IRELAND" : "UNITED KINGDOM",
    "TÜRKIYE": "Turkey",
    "CZECHIA": "CZECH REPUBLIC",
    "SYRIAN ARAB REPUBLIC" : "SYRIA",
}
continent["Country"] = continent["Country"].replace(manualMapping)
# df = pd.merge(df, continent[["Country", "region"]], on="Country", how ="left")
# df.info()

We have filled all almost all the rows.

In [842]:
fillers = {
    "Country": ["VIETNAM", "KOSOVO"],
    "region": ["Asia", "Europe"]
}
fillers = pd.DataFrame(fillers)
continent = pd.concat([continent, fillers], ignore_index=True)
df = pd.merge(df, continent[["Country", "region"]], on="Country", how ="left")
df = df.drop_duplicates()
df["Ranking"] = df["Ranking"].fillna(2001)
df.drop(columns="CollegeRank", errors="ignore").to_csv("../LLM_generated/indexLLM.csv", index=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 434 entries, 0 to 439
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Country           434 non-null    object 
 1   University        434 non-null    object 
 2   CollegeRank       434 non-null    object 
 3   Tuition           434 non-null    int64  
 4   PercOfIntStud     434 non-null    int64  
 5   AcceptanceRate    434 non-null    int64  
 6   SafetyIndex       434 non-null    int64  
 7   LivingCost        434 non-null    int64  
 8   RentCost          434 non-null    int64  
 9   GroceriesCost     434 non-null    int64  
 10  RecreationCost    434 non-null    int64  
 11  HealthcareCost    434 non-null    int64  
 12  TransportCost     434 non-null    int64  
 13  Link              434 non-null    object 
 14  Computer Science  434 non-null    int64  
 15  Business          434 non-null    int64  
 16  Economics         434 non-null    int64  
 17  Ps

In [843]:
df.to_csv("../LLM_generated/pricesLLM.csv", index=False)
df.to_excel("../LLM_generated/pricesLLM.xlsx", index=False)

## Index data set

In [844]:
refIndex = 100

refRecreationCost = round(float(df.loc[df["University"] == "ETH Zurich", "RecreationCost"]),1)
refHealthcareCost = round(float(df.loc[df["University"] == "ETH Zurich", "HealthcareCost"]),1)
refTransportCost = round(float(df.loc[df["University"] == "ETH Zurich", "TransportCost"]),1)
refLivingCost = round(float(df.loc[df["University"] == "ETH Zurich", "LivingCost"]),1)
refRentCost = round(float(df.loc[df["University"] == "ETH Zurich", "RentCost"]),1)
refGroceriesCost = round(float(df.loc[df["University"] == "ETH Zurich", "GroceriesCost"]),1)
print(refGroceriesCost, refHealthcareCost, refLivingCost, refRecreationCost, refRentCost, refTransportCost)

350.0 800.0 2000.0 200.0 1200.0 90.0


In [845]:
df["RecreationIndex"] = ((df["RecreationCost"] * refIndex) / refRecreationCost).round(1)
df["HealthcareIndex"] = ((df["HealthcareCost"] * refIndex) / refHealthcareCost).round(1)
df["TransportIndex"] = ((df["TransportCost"] * refIndex) / refTransportCost).round(1)
df["LivingCostIndex"] = ((df["LivingCost"] * refIndex) / refLivingCost).round(1)
df["RentIndex"] = ((df["RentCost"] * refIndex) / refRentCost).round(1)
df["GroceriesIndex"] = ((df["GroceriesCost"] * refIndex) / refGroceriesCost).round(1)
# df.head(10)

In [846]:
excludeColumns = ["RecreationCost", "HealthcareCost", "TransportCost", "LivingCost", "RentCost", "GroceriesCost"]
df.drop(columns=excludeColumns, errors="ignore").to_csv("../LLM_generated/indexLLM.csv", index=False)
df.drop(columns=excludeColumns, errors="ignore").to_excel("../LLM_generated/indexLLM.xlsx", index=False)